# 数据挖掘-预测贷款用户是否逾期

## Task1：数据分析

#### 数据说明：
数据集是金融数据（非原始数据，已经处理过了），预测贷款用户是否会逾期

表格中 "status" 是结果标签：0表示未逾期，1表示逾期

### 1 数据类型的分析 

#### ● 导入宏包

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns  #Seaborn是基于matplotlib的图形可视化python包
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#作图默认无法显示中文（变成小方格）
import matplotlib as mpl
mpl.rcParams['font.sans-serif']=[u'SimHei']    #设置中文黑体
mpl.rcParams['axes.unicode_minus']=False       #更改字体导致显示不出正负号，设置这个显示正常

#### ● 导入数据

In [5]:
data = pd.read_csv('F:/project/data.csv',engine = 'python')
data.head()

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0
